<a href="https://colab.research.google.com/github/bradymiller2310/FantasyFootballDashboard/blob/main/ESPN_FF_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install espn-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Uninstalling urllib3-2.3.0:
      Successfully uninstalled urllib3-2.3.0


In [3]:
from espn_api.football import League

In [4]:
league = League(league_id=1292514, year=2024, espn_s2="AECPNWyLvfnlcFfE0Xb73O%2BP5ZU9n1G%2FSnP3ixxvuqSwQRcQe7bEQfBuz3YBNeRluTYofdhX2aTajj0jZSF%2B8i9ewdk8Dwf6p9G215F2FjmI06wPKyhjleL4WzaT3PCiP15FDXe55YNMkQeJXgbsCuK1VHbdQ1Nxidq0JxAHezRn2zAt6WwNtSqrF0k2nNHV33VpJqPmMk4msFmaoCaz0UAp95xxy1YmXyEUnZWgA60HJkbsgYkZ8AaESwFENsPYdUvKYqCwnZLjvgI5varhOTDYekjvokdtaCoxJfzOdGLJ3g%3D%3D", swid="{D68FE5F2-46CE-421C-B248-DDA01360BB34}")
print(league.teams)  #Teams in league

[Team(Fightin' Furries), Team(Quon Solo), Team(Jen-eral ⚔️), Team(Captain Sweatpants), Team(Salmon LipBalm !!), Team(Graham’s Groupie), Team(pop-pop's bible study), Team(Bucktown Bandits), Team(bird gang), Team(bungalicious  💅)]


In [7]:
for week in range(1, 15):
    print(f"Week {week} Schedule:")
    for matchup in league.scoreboard(week=week):
        print(f"{matchup.home_team} vs {matchup.away_team}")

Week 1 Schedule:
Team(Bucktown Bandits) vs Team(Graham’s Groupie)
Team(bungalicious  💅) vs Team(Fightin' Furries)
Team(bird gang) vs Team(Salmon LipBalm !!)
Team(pop-pop's bible study) vs Team(Quon Solo)
Team(Jen-eral ⚔️) vs Team(Captain Sweatpants)
Week 2 Schedule:
Team(Fightin' Furries) vs Team(Salmon LipBalm !!)
Team(Quon Solo) vs Team(Bucktown Bandits)
Team(Captain Sweatpants) vs Team(Graham’s Groupie)
Team(pop-pop's bible study) vs Team(bungalicious  💅)
Team(Jen-eral ⚔️) vs Team(bird gang)
Week 3 Schedule:
Team(Quon Solo) vs Team(Captain Sweatpants)
Team(Fightin' Furries) vs Team(pop-pop's bible study)
Team(Salmon LipBalm !!) vs Team(Jen-eral ⚔️)
Team(Bucktown Bandits) vs Team(bungalicious  💅)
Team(Graham’s Groupie) vs Team(bird gang)
Week 4 Schedule:
Team(pop-pop's bible study) vs Team(Jen-eral ⚔️)
Team(bungalicious  💅) vs Team(Quon Solo)
Team(bird gang) vs Team(Captain Sweatpants)
Team(Bucktown Bandits) vs Team(Fightin' Furries)
Team(Graham’s Groupie) vs Team(Salmon LipBalm !!)


In [19]:
free_agents = league.free_agents(size=20, position='QB')  # Get top 20 available QBs

print("Available Free Agents (QB):")
for player in free_agents:
    print(f"  - {player.name} ({player.proTeam})")

Available Free Agents (QB):
  - C.J. Stroud (HOU)
  - Tua Tagovailoa (MIA)
  - Anthony Richardson (IND)
  - Aaron Rodgers (NYJ)
  - Kirk Cousins (ATL)
  - Bryce Young (CAR)
  - Michael Penix Jr. (ATL)
  - Trevor Lawrence (JAX)
  - Jameis Winston (CLE)
  - Derek Carr (NO)
  - Justin Fields (PIT)
  - Joe Flacco (IND)
  - Deshaun Watson (CLE)
  - Cooper Rush (DAL)
  - Daniel Jones (MIN)
  - Tom Brady (TB)
  - Mason Rudolph (TEN)
  - Mac Jones (JAX)
  - Tommy DeVito (NYG)
  - Aidan O'Connell (LV)


In [17]:
for team in league.teams:
    print(f"\nTeam: {team.team_name}")
    print("Roster:")
    for player in team.roster:
        print(f"  - {player.name} ({player.position})")


Team: Fightin' Furries
Roster:
  - Tyreek Hill (WR)
  - Isiah Pacheco (RB)
  - Joe Mixon (RB)
  - Patrick Mahomes (QB)
  - Dalton Kincaid (TE)
  - Zay Flowers (WR)
  - Jaxon Smith-Njigba (WR)
  - Gus Edwards (RB)
  - Matthew Stafford (QB)
  - Kareem Hunt (RB)
  - Broncos D/ST (D/ST)
  - David Njoku (TE)
  - Caleb Williams (QB)
  - DeAndre Hopkins (WR)
  - Bills D/ST (D/ST)
  - Matthew Wright (K)

Team: Quon Solo
Roster:
  - Justin Jefferson (WR)
  - Kyren Williams (RB)
  - Marvin Harrison Jr. (WR)
  - Mike Evans (WR)
  - D'Andre Swift (RB)
  - Brian Robinson Jr. (RB)
  - Kyler Murray (QB)
  - J.K. Dobbins (RB)
  - Geno Smith (QB)
  - Chris Boswell (K)
  - Romeo Doubs (WR)
  - Mark Andrews (TE)
  - Bears D/ST (D/ST)
  - Courtland Sutton (WR)
  - Josh Downs (WR)
  - Evan Engram (TE)

Team: Jen-eral ⚔️
Roster:
  - Derrick Henry (RB)
  - A.J. Brown (WR)
  - Alvin Kamara (RB)
  - Jalen Hurts (QB)
  - Calvin Ridley (WR)
  - Raheem Mostert (RB)
  - Jets D/ST (D/ST)
  - Pat Freiermuth (TE)
  

In [20]:
week_number = 1
box_scores = league.box_scores(week=week_number)

for matchup in box_scores:
    print(f"\nMatchup: {matchup.home_team} vs {matchup.away_team}")
    print(f"  Home Score: {matchup.home_score}")
    print(f"  Away Score: {matchup.away_score}")


Matchup: Team(Bucktown Bandits) vs Team(Graham’s Groupie)
  Home Score: 73.0
  Away Score: 96.0

Matchup: Team(bungalicious  💅) vs Team(Fightin' Furries)
  Home Score: 101.0
  Away Score: 94.0

Matchup: Team(bird gang) vs Team(Salmon LipBalm !!)
  Home Score: 59.0
  Away Score: 101.0

Matchup: Team(pop-pop's bible study) vs Team(Quon Solo)
  Home Score: 79.0
  Away Score: 97.0

Matchup: Team(Jen-eral ⚔️) vs Team(Captain Sweatpants)
  Home Score: 82.0
  Away Score: 110.0
